In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# paths (workspace)
sales_path = "/workspaces/TeamCPH/umsatzdaten_gekuerzt.csv"
wetter_path = "/workspaces/TeamCPH/wetter.csv"
kiwo_path = "/workspaces/TeamCPH/kiwo.csv"

# load sales and aggregate to daily total
sales = pd.read_csv(sales_path, parse_dates=["Datum"])
sales_daily = sales.groupby("Datum", as_index=False)["Umsatz"].sum()

# load weather and kieler-week flags
wetter = pd.read_csv(wetter_path, parse_dates=["Datum"])
kiwo = pd.read_csv(kiwo_path, parse_dates=["Datum"])

# merge: sales <- weather <- kiwo
merged = sales_daily.merge(wetter, on="Datum", how="left")
merged = merged.merge(kiwo, on="Datum", how="left")

# fill missing KielerWoche as 0 (not festival) and sort
merged["KielerWoche"] = merged["KielerWoche"].fillna(0).astype(int)
merged = merged.sort_values("Datum").reset_index(drop=True)

# extract year-month and aggregate
merged["YearMonth"] = merged["Datum"].dt.to_period("M")

# group by month: sum revenue and count KielerWoche days
monthly = merged.groupby("YearMonth").agg({
    "Umsatz": "sum",
    "KielerWoche": "sum"
}).reset_index()

# convert YearMonth back to timestamp for plotting
monthly["YearMonth"] = monthly["YearMonth"].dt.to_timestamp()

# create bar chart
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(monthly["YearMonth"], monthly["Umsatz"], color="steelblue", edgecolor="black")

# add KielerWoche count labels above bars
for i, (bar, kiwo_count) in enumerate(zip(bars, monthly["KielerWoche"])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f"{int(kiwo_count)}",
            ha="center", va="bottom", fontsize=9, fontweight="bold")

# formatting
ax.set_xlabel("Month", fontsize=12)
ax.set_ylabel("Total Revenue (Umsatz)", fontsize=12)
ax.set_title("Monthly Revenue with KielerWoche Days", fontsize=14, fontweight="bold")
ax.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# result
print(merged.head())
# optionally save
# merged.to_csv("/workspaces/TeamCPH/merged_daily.csv", index=False)

       Datum       Umsatz  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0 2013-07-01  1269.249107         6.0     17.8375                 15.0   
1 2013-07-02  1430.008397         3.0     17.3125                 10.0   
2 2013-07-03  1124.274894         7.0     21.0750                  6.0   
3 2013-07-04  1184.309567         7.0     18.8500                  7.0   
4 2013-07-05  1288.023060         5.0     19.9750                 12.0   

   Wettercode  KielerWoche  
0        20.0            0  
1         NaN            0  
2        61.0            0  
3        20.0            0  
4         NaN            0  
